In [1]:
!pip install git+https://github.com/softlab-unimore/fairnesseval@main

  Cloning https://github.com/softlab-unimore/fairnesseval (to revision main) to /tmp/pip-req-build-97y33tbc
  Running command git clone --filter=blob:none --quiet https://github.com/softlab-unimore/fairnesseval /tmp/pip-req-build-97y33tbc
  Resolved https://github.com/softlab-unimore/fairnesseval to commit 286fd102ade92d89c90c5ed542d4562b51d4a413
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/MiroDudik/fairlearn.git (to revision f11f99acd2ba724f9ef885973830d8f3390b0717) to /tmp/pip-install-m6saa6rc/fairlearn_be447643532b4a1e9f30e107bcbe8ddb
  Running command git clone --filter=blob:none --quiet https://github.com/MiroDudik/fairlearn.git /tmp/pip-install-m6saa6rc/fairlearn_be447643532b4a1e9f30e107bcbe8ddb
  Running command git rev-parse -q --verify 'sha^f11f99acd2ba724f9ef885973830d8f3390b0717'
  Running command git fetch -q https://github.com/MiroDudik/fairlearn.git f11f99acd2ba724f9ef885973830d8f3390b0717
  Running command git checkout -q f11f99acd2ba724f9ef8859

### Dataset download
Download the dataset following the instructions in aif360 errors.
The following code should be changed according to your paths (python path especially).

In [3]:
# @title Code to download datasets in colab
import requests
import os

url_list = ["https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv",
            'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data',
	'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc',
    'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
	'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test',
	'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names',
            ]
compas = "/usr/local/lib/python3.10/dist-packages/aif360/data/raw/compas"
german = "/usr/local/lib/python3.10/dist-packages/aif360/data/raw/german"
adult =  "/usr/local/lib/python3.10/dist-packages/aif360/data/raw/adult"
save_path_list = [compas] + [german] *2 + [adult] * 3

for url, save_path in zip(url_list, save_path_list):
    os.makedirs(save_path, exist_ok=True)
    full_path = os.path.join(save_path, url.split('/')[-1])
    response = requests.get(url)
    with open(full_path, 'wb') as file:
        file.write(response.content)

import os
os.makedirs('cached_data/2018/1-Year', exist_ok=True)


In [8]:
# @title Code to download datasets in colab
import requests
import os

url_list = ["https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv",
            'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data',
	'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc',
    'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
	'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test',
	'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names',
            ]
compas = "/usr/local/lib/python3.10/dist-packages/aif360/data/raw/compas"
german = "/usr/local/lib/python3.10/dist-packages/aif360/data/raw/german"
adult =  "/usr/local/lib/python3.10/dist-packages/aif360/data/raw/adult"
save_path_list = [compas] + [german] *2 + [adult] * 3

for url, save_path in zip(url_list, save_path_list):
    os.makedirs(save_path, exist_ok=True)
    full_path = os.path.join(save_path, url.split('/')[-1])
    response = requests.get(url)
    with open(full_path, 'wb') as file:
        file.write(response.content)

import os
os.makedirs('cached_data/2018/1-Year', exist_ok=True)


In [6]:
print("🔁 Restarting kernel...")
# get_ipython().kernel.do_shutdown(True)
quit()

🔁 Restarting kernel...


In [2]:
# @title Experiment Definition and execution
# !pip install ipywidgets
from IPython.utils import io
import warnings
warnings.filterwarnings("ignore")
with io.capture_output() as captured:
    %load_ext autoreload
    %autoreload 2
    !jupyter nbextension enable --py widgetsnbextension

    from fairnesseval import utils_experiment_parameters as exp_params

    import ipywidgets as widgets
    from ipywidgets import interact, interact_manual, Layout
    from ipywidgets import TwoByTwoLayout
    %matplotlib inline
    import os
    import pandas as pd
    import numpy as np
    from fairnesseval.graphic  import utils_results_data
all_datasets = exp_params.ACS_dataset_names+ ['adult', 'compas', 'german', ]
# import models.models as exp_models
# model_list = exp_models.model_list
model_list = [
 'LogisticRegression',
 'expgrad',
 'ThresholdOptimizer',
 'ZafarDI',
#  'fairlearn_full',
 'ZafarEO',
 'Feld']
from fairnesseval.run import launch_experiment_by_config
import logging, sys
import io
res_dir = './demo_results'


experiment_id = widgets.Text(description='Experiment Code:', style={'description_width': 'initial'}, value='demo.default.test')

title_a = widgets.HTML(value="<b>Dataset Selection:</b>")
dataset_widget = widgets.SelectMultiple(options=all_datasets, value=[all_datasets[-3]],
                           layout=widgets.Layout(width='50%', height='auto'),
                           rows=len(all_datasets))

title_b = widgets.HTML(value="<b>Model Selection:</b>")
model_widget = widgets.SelectMultiple(options=model_list, value=[model_list[0]],
                           layout=widgets.Layout(width='50%', height='auto'),
                           rows=len(model_list))


model_parameters_input = widgets.Textarea(placeholder='Enter parameters as key-value pairs e.g.:\n{"param1": value1, "param2": value2}',
                                          layout=widgets.Layout(width='50%', height='auto'))

train_fractions = widgets.Textarea(description='Train fractions:', placeholder='Enter parameters as list pairs e.g.:\n[0.016, 0.063, 0.251, 1.]',
                                          layout=widgets.Layout(width='50%', height='auto'), style={'description_width': 'initial'})

# title_c = widgets.HTML(value="<b>Model Selection:</b>")
c = widgets.IntText(description='Random seed:', style={'description_width': 'initial'},layout=widgets.Layout(width='50%', height='auto'),)

# Button to gather input
button = widgets.Button(description="Run experiment")

# Output area for displaying the gathered input
output = widgets.Output()

def eval_none(x):
    try:
        return eval(x)
    except Exception as e :
        return ''

def on_button_click(button):
    experiment_conf = {
        'experiment_id': experiment_id.value,
        'dataset_names': dataset_widget.value,
        'model_names': model_widget.value,
        'random_seeds': [c.value],
        'model_params': eval_none(model_parameters_input.value),
        'train_fractions': eval_none(train_fractions.value),
        'results_path': res_dir,
        'params': ['--debug'], # TODO remove
    }
    experiment_conf = {key: value for key, value in experiment_conf.items() if value != ''}
    print(experiment_conf)

    try:
        launch_experiment_by_config(experiment_conf)
    except Exception as e:
        raise e

# Attach the click event handler to the button
button.on_click(on_button_click)



# output_widget = widgets.Output(layout=widgets.Layout(width='600px', height='160px', border='solid'))

# class OutputWidgetHandler(logging.StreamHandler):
#     def __init__(self, widget):
#         super().__init__(self)
#         self.widget = widget

#     def emit(self, record):
#         with self.widget:
#             print(self.format(record))

# # Configure logging to use our custom handler
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# logger.addHandler(OutputWidgetHandler(output_widget))

# def custom_print(*args, **kwargs):
#     with output_widget:
#         print(*args, **kwargs)

# Redirect print statements to our custom print function
# old_print = print
# sys.stdout.write = custom_print


# Set the width and height of the VBox
# vbox.layout.width = '800px'
# vbox.layout.height = '1000px'


# Arrange the widgets vertically
vbox = widgets.VBox([
    experiment_id,
    title_a,
    dataset_widget,
    title_b,
    model_widget,
    widgets.HTML("<b>Model Parameters:</b>"),
    model_parameters_input,
    # widgets.HTML("<b>Train fractions:</b>"),
    train_fractions,
    c,
    button,
    # output_widget
])

display(vbox)  # Display the widgets

pip install 'aif360[inFairness]'


17/04/24 10:16:19 INFO:root: Parameters of experiment demo.default.test
{"experiment_id": "demo.default.test",
	"dataset_names": ["adult"],
	"model_names": ["LogisticRegression"],
	"random_seeds": [0],
	"results_path": "./demo_results",
	"params": ["--debug"]}
17/04/24 10:16:19 INFO:root: Started logging.


{'experiment_id': 'demo.default.test', 'dataset_names': ('adult',), 'model_names': ('LogisticRegression',), 'random_seeds': [0], 'results_path': './demo_results', 'params': ['--debug']}


17/04/24 10:16:20 INFO:root: Starting combination: base model: None, dataset_name: adult, model_name: LogisticRegression


Configuration:
dataset_name: ['adult']
model_name: ['LogisticRegression']
results_path: ./demo_results
metrics: default
preprocessing: default
eps: [None]
constraint_code: None
train_fractions: [1]
expgrad_fractions: None
grid_fractions: None
exp_grid_ratio: None
exp_subset: True
save: True
random_seeds: [0]
train_test_seeds: [None]
test_size: 0.3
split_strategy: StratifiedKFold
train_test_fold: [0, 1, 2]
run_linprog_step: True
redo_tuning: False
redo_exp: False
states: None
base_model_code: None
experiment_id: demo.default.test
model_params: {}
debug: True
num_data_points: None
num_features: None
theta: 0.5
groups: None
group_prob: None
y_prob: None
switch_pos: None
switch_neg: None
****************************************************************************************************


...

Saving results in: ./demo_results/demo.default.test/demo.default.test_adult.csv



17/04/24 10:16:22 INFO:root: Ended fit:  in: 0:00:00.437635 ||| Starting evaluation...
17/04/24 10:16:23 INFO:root: Ended evaluation:  in: 0:00:00.415570
17/04/24 10:16:23 INFO:root: Ended step in: 0:00:00.881136
2it [00:01,  1.05it/s]17/04/24 10:16:23 INFO:root: Starting step: random_seed: 2, train_test_seed: 0, train_test_fold: 2 
{"eps": null, "train_fractions": 1}
17/04/24 10:16:23 INFO:root: Starting fit...


Saving results in: ./demo_results/demo.default.test/demo.default.test_adult.csv



17/04/24 10:16:23 INFO:root: Ended fit:  in: 0:00:00.399371 ||| Starting evaluation...
17/04/24 10:16:24 INFO:root: Ended evaluation:  in: 0:00:00.404202
17/04/24 10:16:24 INFO:root: Ended step in: 0:00:00.832270
3it [00:02,  1.07it/s]
17/04/24 10:16:24 INFO:root: Ended: None, dataset_name: adult, model_name: LogisticRegression in:
 0:00:03.977187
17/04/24 10:16:24 INFO:root: Ended experiment. It took: 0:00:04.197789


Saving results in: ./demo_results/demo.default.test/demo.default.test_adult.csv


In [ ]:
# example of model parameters for fairlearn
# {'eps': [0.001, .005, 0.01, 0.02, 0.05, 0.10, 0.15], 'constraint_code': 'dp'}

# demo.1.test
# train fractions
[0.016, 0.063, 0.251, 1.]
# model params
{'eps': 0.005, 'constraint_code': 'dp', 'base_model_code': ['lr']}
{'base_model_code': ['lr']}
pass

# Presentation

In [3]:
# @title Presentation single dataset

from fairnesseval.graphic import utils_results_data
from fairnesseval.graphic.graphic_utility import plot_demo_subplots
import matplotlib.pyplot as plt

import logging

# Disable fontTools logging
logging.getLogger('fontTools.subset').setLevel(logging.ERROR)

from fairnesseval.graphic.graphic_demoB import plot_function_B


    # return table_res

# plot_function_B(chart_name='D', experiment_code_list=['demo.0.test','demo.1.test',], model_list=['fairlearn','LogisticRegression'], x_axis='train_fractions', y_axis_list=['test_error', 'test_DemographicParity', 'time'])

# Define a layout for the widgets
widget_layout = widgets.Layout(width='95%', height='auto')

# Define a style for the widgets
widget_style = {'description_width': '200px'}
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='flex-start', width='50%')

experiment_list = sorted(utils_results_data.available_experiment_results(res_dir))
experiment_codes_widget = widgets.SelectMultiple(
    options=experiment_list,
    value=[], description='Experiment Codes:',layout=widget_layout, style=widget_style,
    rows=min(5,len(experiment_list)))

name_widget = widgets.Text(description='Chart Name:',value='demo.A',
    layout=widget_layout, style=widget_style)

model_widget = widgets.SelectMultiple(options=[''],
    description='Models to display:',
    layout=widget_layout, style=widget_style)

x_axis_widget = widgets.Dropdown(options=[''],description='X-Axis Attribute:',
    value=None,layout=widget_layout, style=widget_style)

y_axis_widget = widgets.Dropdown(options=[''],description='Y-Axis Attributes:',
    layout=widget_layout, style=widget_style)

dataset_widget = widgets.Dropdown(options=[''],description='Select Dataset:',
    value=None,layout=widget_layout, style=widget_style)

grouping_col_widget = widgets.Dropdown(options=[''],
    description='Group by Attribute:',value=None,
    layout=widget_layout, style=widget_style)


def update_inputs(selected_codes):
    try:
        results_df = utils_results_data.load_results_experiment_id(selected_codes['new'], res_dir)
    except Exception as e:
        # raise e
        print(e)
        return
    model_list = results_df['model_code'].unique()
    model_widget.options = model_list
    model_widget.rows = min(len(model_list), 5)
    model_widget.value = tuple(model_list)
    cols = results_df.columns.tolist()
    grouping_col_widget.options = [None] + cols
    x_axis_widget.options = cols
    y_axis_widget.options = cols
    dataset_widget.options = results_df['dataset_name'].unique()  # Update dataset options

output = widgets.Output()

def generate_graph(chart_name, experiment_code_list, model_list, x_axis, y_axis_list, grouping_col, dataset_list):
    y_axis_list= [y_axis_list]
    dataset_list = [dataset_list]
    params = {key: value for key,value in locals().items()}
    # print(params)
    for key,value in params.items():
        if key in ['grouping_col']:
            continue
        if value is None or len(value) < 1:
            print(f'{key} is empty')
            return
    try:
        fig = plot_function_B(**params, res_path=res_dir, single_plot=False, show=False)
        plt.show()
    except Exception as e:
        # raise e
        print(e)
        pass


experiment_codes_widget.observe(update_inputs, 'value')

# Display widgets in a VBox
widgets_vbox = widgets.VBox([
    name_widget,
    experiment_codes_widget,
    dataset_widget,
    model_widget,
    x_axis_widget,
    y_axis_widget,
    grouping_col_widget,
], layout=box_layout)

output = widgets.interactive_output(
    generate_graph,
    {
        'chart_name': name_widget,
        'experiment_code_list': experiment_codes_widget,
        'model_list': model_widget,
        'x_axis': x_axis_widget,
        'y_axis_list': y_axis_widget,
        'grouping_col': grouping_col_widget,
        'dataset_list': dataset_widget
    }
)
output.layout = widgets.Layout(width='50%', height='auto')

display(widgets_vbox, output)

Output(layout=Layout(height='auto', width='50%'))

In [5]:
# @title Presentation multiple plots
import ipywidgets as widgets
from ipywidgets import interactive_output
from IPython.display import clear_output, display
# Define a layout for the widgets
widget_layout = widgets.Layout(width='95%', height='auto')

# Define a style for the widgets
widget_style = {'description_width': '200px'}
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='flex-start', width='50%')

experiment_list = sorted(utils_results_data.available_experiment_results(res_dir))
experiment_codes_widget = widgets.SelectMultiple(
    options=experiment_list,
    value=[], description='Experiment Codes:',layout=widget_layout, style=widget_style,
    rows=min(5,len(experiment_list)))

name_widget = widgets.Text(description='Chart Name:',value='demo.B',
    layout=widget_layout, style=widget_style)

model_widget = widgets.SelectMultiple(options=[''],
    description='Models to display:',
    layout=widget_layout, style=widget_style)

x_axis_widget = widgets.Dropdown(options=[''],description='X-Axis Attribute:',
    value=None,layout=widget_layout, style=widget_style)

y_axis_widget = widgets.SelectMultiple(options=[''],description='Y-Axis Attributes:',
    layout=widget_layout, style=widget_style)

generate_button = widgets.Button(description='Generate Graph',
    disabled=False,button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Generate Graph', icon='check', # (FontAwesome names without the `fa-` prefix)
    layout=widget_layout)

grouping_col_widget = widgets.Dropdown(options=[''],
    description='Group by Attribute:',value=None,
    layout=widget_layout, style=widget_style)


def update_inputs(selected_codes):
    # print(selected_codes)
    try:
        results_df = utils_results_data.load_results_experiment_id(selected_codes['new'], res_dir)
    except Exception as e:
        print(e)
        return
    model_list = results_df['model_code'].unique()
    model_widget.options = model_list
    model_widget.rows = min(len(model_list), 5)
    model_widget.value = tuple(model_list)
    cols = results_df.columns.tolist()
    grouping_col_widget.options = [None] + cols
    x_axis_widget.options = cols
    y_axis_widget.rows = min(len(cols), 8)
    y_axis_widget.options = cols

def generate_graph(chart_name, experiment_code_list, model_list, x_axis, y_axis_list, grouping_col):
    params = {key: value for key,value in locals().items()}
    # print(params)
    for key,value in params.items():
        if key in ['grouping_col']:
            continue
        if value is None or len(value) < 1:
            # print(f'{key} is empty')
            return
    try:
        clear_output(wait=True)  # Clear the output
        # print('plotting')

        # fig, ax = plt.subplots(figsize=(6, 4))
        # ax.plot(range(len(params['model_list'])))
        fig = plot_function_B(**params, res_path=res_dir, show=False)
        plt.show()
        # display(widgets_vbox, output)
        # display(out)
    except Exception as e:
        # raise e
        print(e)
        pass

# Step 7: Update graph when parameters change
def parameters_changed(change):
    generate_graph(name_widget.value, experiment_codes_widget.value, model_widget.value,
                   x_axis_widget.value, y_axis_widget.value, grouping_col_widget.value)

experiment_codes_widget.observe(update_inputs, 'value')


# Display widgets in a VBox
widgets_vbox = widgets.VBox([
    name_widget,
    experiment_codes_widget,
    model_widget,
    x_axis_widget,
    y_axis_widget,
    grouping_col_widget
], layout=box_layout)

output = interactive_output(
    generate_graph,
    {
        'chart_name': name_widget,
        'experiment_code_list': experiment_codes_widget,
        'model_list': model_widget,
        'x_axis': x_axis_widget,
        'y_axis_list': y_axis_widget,
        'grouping_col': grouping_col_widget
    }
)
output.layout = widgets.Layout(width='50%', height='auto')

display(widgets_vbox, output)

Output(layout=Layout(height='auto', width='50%'))